In [1]:
import socket
import cv2
import pickle
import struct
import imutils
import numpy as np
from mss import mss
import socket
from pynput.mouse import Button, Controller
import pyautogui
from threading import Thread
import multiprocessing
import ray

ray.init()

host_name  = socket.gethostname()
host_ip = socket.gethostbyname(host_name)
print('HOST IP:',host_ip)

@ray.remote
def HostScreenCapturing():
    HostScreenCapturingSocket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    HostScreenCapturingName  = socket.gethostname()
    HostScreenCapturingIP = socket.gethostbyname(HostScreenCapturing)
    print('HOST IP:',HostScreenCapturingIP)
    HostScreenCapturingPort = 9999
    socket_address = (HostScreenCapturingIP,HostScreenCapturingPort)
    HostScreenCapturingSocket.bind(socket_address)
    HostScreenCapturingSocket.listen(5)
    print("LISTENING AT:", socket_address)

    ClientScreenCapturingSocket, ClientScreenCapturingAddr = HostScreenCapturingSocket.accept()
    print('GOT CONNECTION FROM:', ClientScreenCapturingAddr)

    screen_width, screen_height = pyautogui.size()
    frame_size = (screen_width, screen_height)

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter('output.mp4', fourcc, 15.0, frame_size)

    while True:
        img = pyautogui.screenshot()
        frame = np.array(img)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        out.write(frame)

        encoded_frame = frame.tobytes()
        frame_size = len(encoded_frame)
        ClientScreenCapturingSocket.sendall(frame_size.to_bytes(4, 'big') + encoded_frame)
        if cv2.waitKey(1) == ord('q'):
            break
        
    out.release()
    cv2.destroyAllWindows()
    ClientScreenCapturingSocket.close()
    HostScreenCapturingSocket.close()

@ray.remote
def HostMouseCourseControlling():
    Mouse_Coruse_Server_Socket = socket.socket(socket.AF_INET,socket.SOCK_STREAM)

    print('HOST IP:',host_ip)
    Mouse_Coruse_Port = 9922
    Mouse_Coruse_Server_Address = (host_ip, Mouse_Coruse_Port)
    Mouse_Coruse_Server_Socket.bind(Mouse_Coruse_Server_Address)
    Mouse_Coruse_Server_Socket.listen(1)
    print("LISTENING AT:", Mouse_Coruse_Server_Address)
    Mouse_Coruse_Client_Socket, Mouse_Coruse_Client_Address = Mouse_Coruse_Server_Socket.accept()

    screenW, screenH = pyautogui.size()
    midX = screenW // 2
    midY = screenH // 2
    mouse = Controller()
    mouse.position = (midX,midY)

    def get_button_from_string(button_str):
        if button_str == 'Button.left':
            return Button.left
        elif button_str == 'Button.right':
            return Button.right
        elif button_str == 'Button.middle':
            return Button.middle
        else:
            return Button.unknown

    try:
        while True:
            data = Mouse_Coruse_Client_Socket.recv(1024).decode()
            if not data:
                break
            events = data.split('\n')
            for event in events:
                event_parts = event.split()
                if len(event_parts) < 2:
                    continue

                event_type = event_parts[0]

                if event_type == 'MOVE':
                    if len(event_parts) < 3:
                        continue
                    try:
                        x, y = int(event_parts[1]), int(event_parts[2])
                        mouse.position = (x, y)
                    except ValueError:
                        continue

                elif event_type == 'PRESS' or event_type == 'RELEASE':
                    if len(event_parts) < 4:
                        continue
                    button_str = event_parts[1]
                    button = get_button_from_string(button_str)
                    try:
                        x, y = int(event_parts[2]), int(event_parts[3])
                        if event_type == 'PRESS':
                            mouse.position = (x, y)
                            if button == mouse.scroll:
                                if len(event_parts) < 5:
                                    continue
                                scroll_amount = int(event_parts[4])
                                mouse.scroll(100, scroll_amount)
                            else:
                                mouse.press(button)
                        else:
                            mouse.position = (x, y)
                            if button != mouse.scroll:
                                mouse.release(button)
                    except ValueError:
                        continue
    except KeyboardInterrupt:
        ...
        # pass

    Mouse_Coruse_Client_Socket.close()
    Mouse_Coruse_Server_Socket.close()

c:\Program Files\Python38\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-05-08 19:53:33,235	INFO worker.py:1625 -- Started a local Ray instance.


HOST IP: 192.168.1.120


In [2]:
ray.get([HostScreenCapturing.remote(), HostMouseCourseControlling.remote()])

(HostScreenCapturing pid=9544) LISTENING AT: ('192.168.1.120', 9999)
(HostMouseCourseControlling pid=17144) HOST IP: 192.168.1.120
(HostScreenCapturing pid=9544) GOT CONNECTION FROM: ('192.168.1.126', 1043)
(HostMouseCourseControlling pid=17144) GOT CONNECTION FROM: ('192.168.1.126', 1043)


RayTaskError(ConnectionResetError): [36mray::HostScreenCapturing()[39m (pid=9544, ip=127.0.0.1)
  File "python\ray\_raylet.pyx", line 877, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 881, in ray._raylet.execute_task
  File "C:\Users\Magshimim\AppData\Local\Temp\ipykernel_24836\3482764600.py", line 49, in HostScreenCapturing
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

2023-05-08 19:55:42,382	ERROR worker.py:408 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::HostMouseCourseControlling() (pid=17144, ip=127.0.0.1)
  File "python\ray\_raylet.pyx", line 877, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 881, in ray._raylet.execute_task
  File "C:\Users\Magshimim\AppData\Local\Temp\ipykernel_24836\3482764600.py", line 88, in HostMouseCourseControlling
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
